In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from collections import defaultdict
import json
import time
import warnings

warnings.filterwarnings(action='ignore')
options = Options()
options.add_argument("start-maximized")
options.add_argument('--disable-gpu')
# options.add_argument('--headless')

In [29]:
# url='https://jasoseol.com/etc/set_user_preference'

# payload={'preference_id': 4, 'value': '210,211,212,213,214,215'}

# res = requests.post(url, data=payload, verify=False)
# print(res.text)

In [2]:
def get_driver():
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

#### 채용캘린더 필터링 및 크롤링

In [14]:
driver = get_driver()
url = 'https://jasoseol.com/recruit' 
driver.implicitly_wait(5)
driver.get(url)

# 조회 조건 - 신입, 인턴, 대기업, 금융
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/input'))).click()
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div/div[2]/div/div[4]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]'))).click()
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div/div[2]/div/div[4]/div[1]/div[1]/div[1]/div[1]/div[2]/div[3]'))).click()
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div/div[2]/div/div[4]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]'))).click()
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div/div[2]/div/div[4]/div[1]/div[2]/div/div/div[2]/div/div[1]/div/div[8]/div[1]'))).click()
# '채용기간 - 시작'
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div/div[2]/div/div[4]/div[1]/div[1]/div[1]/div[3]/div[2]/div[1]'))).click()
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[2]/div/div[2]/div/div[4]/div[2]'))).click()
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div[2]/div/div[3]/div[1]'))).click()

wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div[2]/div/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/img'))).click() #전월
driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div[2]/div/div[3]/div[1]/div[2]'))).click()
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

r_com_single = soup.find_all('div', {'class':'single_employment'})
r_com_single = [r for r in r_com_single if 'hide' not in ''.join(r['class'])]
r_com_list = []
for r in r_com_single:
    r_id = r.find('a', {'class':'company'})['employment_company_id']
    r_com_list.append(r_id)

r_com_grouped = driver.find_elements(by=By.CLASS_NAME, value='grouped_employment')
for r in r_com_grouped:
    r.find_element(by=By.TAG_NAME, value='span').click()
    r_soup = BeautifulSoup(driver.page_source, 'html.parser')
    r_id_list = r_soup.find('div', {'class':'employment-group-item-container_scroll-box'}).find_all('div', {'class':'employment-group-item'})
    for l in r_id_list:
        r_com_list.append(l['employment_id'])
    ActionChains(driver).send_keys(Keys.ESCAPE).perform()

In [15]:
r_com_list

['81473',
 '81735',
 '81710',
 '81846',
 '81928',
 '82000',
 '82291',
 '82449',
 '82454',
 '82490',
 '82689',
 '82999',
 '83100',
 '83127',
 '83288',
 '83254',
 '83237',
 '83397',
 '83306',
 '83413',
 '83426',
 '83536',
 '83479',
 '83551',
 '83267',
 '83270',
 '83271',
 '83225',
 '83261']

#### 상세페이지 크롤링

In [16]:
r_df = pd.DataFrame(columns=['회사명', '종류', '접수시작', '접수마감', '모집부문', 'URL'])

In [17]:
for r in r_com_list:
    url = 'https://jasoseol.com/api/v1/employment_companies/' + r
    response = requests.get(url, verify=False)
    res_json = json.loads(response.content)
    r_name = res_json['name']
    r_start = res_json['start_time']
    r_end = res_json['end_time']
    r_emp = res_json['employments']
    r_field = []
    r_division = [] # 1 신입, 2 경력, 3 인턴
    for f in r_emp:
        r_field.append(f['field'])
        r_division.append(f['division'])
        
    r_dict = {}
    r_division = [[str(x) for x in lst] for lst in r_division]
    for idx in range(len(r_division)):
        key = ','.join(r_division[idx])
        if key in r_dict.keys():
            r_dict[key].append(r_field[idx])
        else:
            r_dict[key] = [r_field[idx]]
            
    for key, value in r_dict.items():
        r_list = [r_name, key, r_start, r_end, ','.join(value), url]
        r_df.loc[len(r_df)] = r_list

In [18]:
r_df

,회사명,종류,접수시작,접수마감,모집부문,URL
0,롯데캐피탈,1,2023-05-02T10:00:00.000+09:00,2023-05-15T23:00:00.000+09:00,"금융영업,신용분석,IT개발,여신관리,경영관리",https://jasoseol.com/api/v1/employment_compani...
1,KCC실리콘,1,2023-05-04T00:00:00.000+09:00,2023-05-11T23:59:00.000+09:00,"채권관리,원가회계,공무-기계,서무",https://jasoseol.com/api/v1/employment_compani...
2,하나생명보험,"1,2",2023-05-04T11:18:00.000+09:00,2023-05-17T23:59:00.000+09:00,상품개발,https://jasoseol.com/api/v1/employment_compani...
3,NH농협캐피탈,1,2023-05-08T07:00:00.000+09:00,2023-05-15T18:00:00.000+09:00,"리스크·여신관리,영업일반(기업금융),영업일반(리테일금융),디지털,IT,전문자격(회계...",https://jasoseol.com/api/v1/employment_compani...
4,캐롯손해보험,3,2023-05-08T12:00:00.000+09:00,2023-05-26T23:59:00.000+09:00,대인 보상,https://jasoseol.com/api/v1/employment_compani...
5,KB증권,1,2023-05-10T09:00:00.000+09:00,2023-05-24T17:00:00.000+09:00,"S&T (Sales & Trading),IB,디지털,IT,PB(수도권),PB(대전권...",https://jasoseol.com/api/v1/employment_compani...
6,롯데손해보험,1,2023-05-11T18:00:00.000+09:00,2023-05-22T18:00:00.000+09:00,"재무경리,자산운용,자동차보험 상품개발,자동차보험 계약관리,장기보상,AI서비스 개발",https://jasoseol.com/api/v1/employment_compani...
7,KB손해사정,1,2023-05-15T10:00:00.000+09:00,2023-05-21T23:59:00.000+09:00,"장기보험심사,재물보험심사",https://jasoseol.com/api/v1/employment_compani...
8,신영증권,3,2023-05-15T09:40:00.000+09:00,2023-06-06T23:00:00.000+09:00,"자산관리서비스,본사영업,운용,리서치,IT,본사관리",https://jasoseol.com/api/v1/employment_compani...
9,푸른저축은행,"1,2",2023-05-15T14:00:00.000+09:00,2023-05-31T23:59:00.000+09:00,금융일반,https://jasoseol.com/api/v1/employment_compani...


#### 전처리

In [22]:
replace_dict = {'1':'신입', '2':'경력', '3':'인턴', '1,2':'신입, 경력', '1,3':'신입, 인턴', '2,3':'경력, 인턴'}
r_df2 = r_df[r_df['종류']!='4'].replace({'종류':replace_dict})
r_df2['접수시작'] = r_df2['접수시작'].str[:10]
r_df2['접수마감'] = r_df2['접수마감'].str[:10]
r_df2 = r_df2.sort_values('접수시작')
r_df2.tail()

,회사명,종류,접수시작,접수마감,모집부문,URL
23,미래에셋증권,경력,2023-05-31,2023-06-12,"Global 사업 기획/관리,Global IT/디지털",https://jasoseol.com/api/v1/employment_compani...
24,이베스트투자증권,인턴,2023-06-01,2023-06-11,"본사지원-리스크관리,영업지원-지점업무",https://jasoseol.com/api/v1/employment_compani...
25,카카오페이,인턴,2023-06-01,2023-06-30,착오송금 중개 및 송금서비스 운영 지원,https://jasoseol.com/api/v1/employment_compani...
26,포스코모빌리티솔루션,"신입, 경력",2023-06-02,2023-06-09,투자 관리,https://jasoseol.com/api/v1/employment_compani...
27,하나은행,인턴,2023-06-02,2023-06-13,"디지털,디자인 크리에이터",https://jasoseol.com/api/v1/employment_compani...


In [23]:
r_df2.to_excel('./jasoseol_0616.xlsx')

In [24]:
# r_df_f = r_df2
# r_df_f.head()

In [25]:
# r_df_f = pd.concat([r_df_f, r_df2])
# r_df_f = r_df_f.drop_duplicates()
# r_df_f

In [77]:
# r_df_f.to_excel('./jasoseol_0613.xlsx')